# Temat: Pali się

https://www.kaggle.com/datasets/gatandubuc/public-transport-traffic-data-in-france

## Cel -> wykrycie sytuacji w której pożar zagraża ludziom

## Stakeholder -> straż pożarna pracująca nad systemem wczesnego ostrzegania oraz reagowania na zagrożenia pożarowe



#### Doczytać o niezbilansowanych zbiorach danych
#### [przegląd różnych metod]
#### Test - zbudować model bez żadnego bilansowania
#### Benchmark - zrobienie minimum i zbudowanie modelu benchmarkowego [usunięcie braków, zamiana zmiennych kategorycznych]
#### Zmienne kategoryczne - zacząć od label encodingu AS-IS, w drugiej kolejności ewentualnie zrobić grupowanie

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc 

import Functions_ML as my_functions

%matplotlib inline

In [11]:
df = pd.read_csv('Fire-Incidents.csv')

pd.set_option('display.max_columns', None)

### Remove unnecessary columns

In [12]:
col_to_drop = ['Business_Impact', 'Estimated_Dollar_Loss', 'Estimated_Number_Of_Persons_Displaced',
                'Ext_agent_app_or_defer_time', 'Last_TFS_Unit_Clear_Time']

df = df.drop(columns = col_to_drop, axis = 1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11214 entries, 0 to 11213
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Area_of_Origin                            11214 non-null  object 
 1   Civilian_Casualties                       11214 non-null  int64  
 2   Count_of_Persons_Rescued                  11214 non-null  int64  
 3   Extent_Of_Fire                            11214 non-null  object 
 4   Fire_Alarm_System_Impact_on_Evacuation    11214 non-null  object 
 5   Fire_Alarm_System_Operation               11214 non-null  object 
 6   Fire_Alarm_System_Presence                11214 non-null  object 
 7   Fire_Under_Control_Time                   11214 non-null  object 
 8   Ignition_Source                           11214 non-null  object 
 9   Incident_Station_Area                     11214 non-null  int64  
 10  Incident_Ward                     

In [13]:
df.isna().sum()

Area_of_Origin                               0
Civilian_Casualties                          0
Count_of_Persons_Rescued                     0
Extent_Of_Fire                               0
Fire_Alarm_System_Impact_on_Evacuation       0
Fire_Alarm_System_Operation                  0
Fire_Alarm_System_Presence                   0
Fire_Under_Control_Time                      0
Ignition_Source                              0
Incident_Station_Area                        0
Incident_Ward                               33
Latitude                                     0
Longitude                                    0
Material_First_Ignited                       0
Method_Of_Fire_Control                       0
Possible_Cause                               0
Property_Use                                 0
Smoke_Alarm_at_Fire_Origin_Alarm_Failure     0
Smoke_Alarm_at_Fire_Origin_Alarm_Type        0
Status_of_Fire_On_Arrival                    0
TFS_Alarm_Time                               0
TFS_Arrival_T

In [14]:
df.dropna(inplace = True)

In [15]:
df['TFS_Alarm_Time'] = pd.to_datetime(df['TFS_Alarm_Time'])
df['TFS_Arrival_Time'] = pd.to_datetime(df['TFS_Arrival_Time'])

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11181 entries, 0 to 11213
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Area_of_Origin                            11181 non-null  object        
 1   Civilian_Casualties                       11181 non-null  int64         
 2   Count_of_Persons_Rescued                  11181 non-null  int64         
 3   Extent_Of_Fire                            11181 non-null  object        
 4   Fire_Alarm_System_Impact_on_Evacuation    11181 non-null  object        
 5   Fire_Alarm_System_Operation               11181 non-null  object        
 6   Fire_Alarm_System_Presence                11181 non-null  object        
 7   Fire_Under_Control_Time                   11181 non-null  object        
 8   Ignition_Source                           11181 non-null  object        
 9   Incident_Station_Area            

In [17]:
df.loc[df['Civilian_Casualties'] >= 1, 'Civilian_Casualties'] = 1

In [18]:
df.head()

,Area_of_Origin,Civilian_Casualties,Count_of_Persons_Rescued,Extent_Of_Fire,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Fire_Under_Control_Time,Ignition_Source,Incident_Station_Area,Incident_Ward,Latitude,Longitude,Material_First_Ignited,Method_Of_Fire_Control,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Status_of_Fire_On_Arrival,TFS_Alarm_Time,TFS_Arrival_Time
0,Porch or Balcony,0,86,Confined to object of origin,Not applicable: Occupant(s) first alerted by o...,Fire alarm system did not operate,Fire alarm system present,2018-08-24T17:06:28,Undetermined,114,18.0,43.760357,-79.412479,Undetermined (formerly 98),Extinguished by fire department,Undetermined,Multi-Unit Dwelling - Over 12 Units,Not applicable: Alarm operated OR presence/ope...,Hardwired (standalone),Fire with smoke showing only - including vehic...,2018-08-24 16:49:36,2018-08-24 16:54:09
1,Cooking Area or Kitchen,0,28,Confined to part of room/area of origin,Not applicable: Occupant(s) first alerted by o...,Fire alarm system operated,Fire alarm system present,2018-11-24T07:19:00,"Other Cooking Items (eg Toaster, Kettle, elec ...",142,7.0,43.754542,-79.530419,Plastic,Extinguished by occupant,Under Investigation,Infirmary,Not applicable: Alarm operated OR presence/ope...,Interconnected,Fire extinguished prior to arrival,2018-11-24 07:09:12,2018-11-24 07:14:23
2,"Living Area (e.g. living, TV, recreation, etc)",1,16,"Spread beyond room of origin, same floor",Some persons (at risk) evacuated as a result o...,Fire alarm system operated,Fire alarm system present,2017-02-09T18:36:21,Undetermined,325,27.0,43.657300,-79.373460,Undetermined (formerly 98),Extinguished by fire department,Undetermined,Multi-Unit Dwelling - Over 12 Units,Not applicable: Alarm operated OR presence/ope...,Hardwired (standalone),Flames showing from small area (one storey or ...,2017-02-09 17:45:07,2017-02-09 17:48:49
3,Undetermined (formerly 98),0,12,Entire Structure,Undetermined,Fire alarm system operation undetermined,Undetermined,2012-10-30T01:28:48,Undetermined,332,20.0,43.649350,-79.393700,Undetermined (formerly 98),Extinguished by fire department,Undetermined,"Clothing Store, Accessories, fur",Not applicable: Alarm operated OR presence/ope...,Type undetermined,Flames showing from large area (more than one ...,2012-10-30 00:42:01,2012-10-30 00:44:58
4,"Sleeping Area or Bedroom (inc. patients room, ...",1,11,"Spread beyond room of origin, same floor",Some persons (at risk) evacuated as a result o...,Fire alarm system operated,Fire alarm system present,2018-07-08T04:38:28,Matches or Lighters (unable to distinguish),442,5.0,43.697960,-79.511539,Bedding,Extinguished by fire department,Suspected Arson,Multi-Unit Dwelling - Over 12 Units,Not applicable: Alarm operated OR presence/ope...,Interconnected,Fire with no evidence from street,2018-07-08 04:08:50,2018-07-08 04:13:54


In [26]:
df['Civilian_Casualties'].value_counts()


Civilian_Casualties
0    10233
1      948
Name: count, dtype: int64

In [27]:
df['Property_Use'].value_counts().head(25)

Property_Use
Multi-Unit Dwelling - Over 12 Units                              3716
Detached Dwelling                                                2309
Semi-Detached Dwelling                                            727
Attached Dwelling (eg. rowhouse, townhouse, etc.)                 577
Multi-Unit Dwelling - 2 to 6 Units                                279
Apartment, Flat, Tenement with Business                           276
Restaurants (occupant load greater than 30 persons, licensed)     229
Restaurant (occupant load less than 30 persons)                   164
Multi-Unit Dwelling - 7 to 12 Units                               145
Detached Garage                                                   134
Subway Station                                                    131
Rooming/Boarding/Lodging House                                    121
General Business Office                                           119
Motor Vehicle Repair Garage                                       113
Other B

### Calculating the time when the fire brigade arrived

In [28]:
df['Response'] = (df['TFS_Arrival_Time'] - df['TFS_Alarm_Time']) / pd.Timedelta(minutes = 1)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11181 entries, 0 to 11213
Data columns (total 23 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Area_of_Origin                            11181 non-null  int64         
 1   Civilian_Casualties                       11181 non-null  int64         
 2   Count_of_Persons_Rescued                  11181 non-null  int64         
 3   Extent_Of_Fire                            11181 non-null  int64         
 4   Fire_Alarm_System_Impact_on_Evacuation    11181 non-null  int64         
 5   Fire_Alarm_System_Operation               11181 non-null  object        
 6   Fire_Alarm_System_Presence                11181 non-null  object        
 7   Fire_Under_Control_Time                   11181 non-null  object        
 8   Ignition_Source                           11181 non-null  int64         
 9   Incident_Station_Area            

### Set 'Property_use' string value 'Business' or 'Non-Business' based on Dwelling